# AI-Powered Deep Learning System for Brain Tumor Prediction and Classification Using MRI Images
# Dr Subramani

# vgg16 model result, Validation Loss: 15.19,  Accuracy: 71.32%

In [1]:
# 1. IMPORT LIBRARIES
# ===================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import time
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(" Device:", device)

# 2. IMAGE TRANSFORMS (with DATA AUGMENTATION)
# ============================================

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.15, contrast=0.15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


# 3. LOAD DATASET
# ===============

train_dir = "mri_dataset_images/train"
test_dir  = "mri_dataset_images/test"

train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset  = datasets.ImageFolder(test_dir, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)


# 4. LOAD PRETRAINED VGG16
# =========================

model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

# Freeze feature layers (FAST training)
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier for 4 classes
model.classifier[6] = nn.Linear(4096, 4)

model = model.to(device)


# 5. LOSS, OPTIMIZER
# ===================

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0001)


# 6. TRAINING FUNCTION + BEST MODEL SAVING + EARLY STOPPING
# ==========================================================

best_loss = np.inf
patience = 5
early_stop_counter = 0

def train_model(epochs=20):
    global best_loss, early_stop_counter

    for epoch in range(1, epochs + 1):
        start = time.time()
        model.train()
        running_loss = 0

        # -------------------------- Training Loop -------------------------
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        # -------------------------- Validation Loop -------------------------
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total

        print(f"\nEpoch {epoch}:")
        print(f"Train Loss: {running_loss:.2f}")
        print(f"Validation Loss: {val_loss:.2f},  Accuracy: {acc:.2f}%")

        # -------------------------- Save Best Model -------------------------
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), "best_brain_tumor_model.pth")
            print(" Best model saved!")
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            print(f" No improvement. Early stopping in {patience - early_stop_counter} epochs.")

        # -------------------------- Early Stopping -------------------------
        if early_stop_counter == patience:
            print("\n Early Stopping Activated!")
            break

        print(f"Time Taken: {time.time() - start:.2f} sec")


# 7. START TRAINING
# ==================

train_model(epochs=20)

print("\n Training Completed!")
print("Best Model Path: best_brain_tumor_model.pth")


💻 Device: cpu
Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

Epoch 1:
Train Loss: 56.74
Validation Loss: 16.51,  Accuracy: 60.15%
💾 Best model saved!
Time Taken: 537.56 sec

Epoch 2:
Train Loss: 33.30
Validation Loss: 19.03,  Accuracy: 63.96%
⏳ No improvement. Early stopping in 4 epochs.
Time Taken: 533.81 sec

Epoch 3:
Train Loss: 25.89
Validation Loss: 15.19,  Accuracy: 71.32%
💾 Best model saved!
Time Taken: 542.24 sec

Epoch 4:
Train Loss: 20.21
Validation Loss: 18.94,  Accuracy: 72.08%
⏳ No improvement. Early stopping in 4 epochs.
Time Taken: 526.46 sec

Epoch 5:
Train Loss: 16.78
Validation Loss: 20.31,  Accuracy: 71.32%
⏳ No improvement. Early stopping in 3 epochs.
Time Taken: 531.22 sec

Epoch 6:
Train Loss: 16.82
Validation Loss: 17.77,  Accuracy: 73.86%
⏳ No improvement. Early stopping in 2 epochs.
Time Taken: 523.57 sec

Epoch 7:
Train Loss: 14.66
Validation Loss: 21.53,  Accuracy: 71.57%
⏳ No improvement. Early stopping in 1 epochs.
Time Taken:

# Denseet121 Model result Val Loss: 1.0570, Accuracy: 70.56%

In [4]:
# DenseNet121 

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import time
import copy

# ------------------------------------------------------------
# DEVICE
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(" Device:", device)

# ------------------------------------------------------------
# PATHS
# ------------------------------------------------------------
train_dir = "mri_dataset_images/train"
val_dir = "mri_dataset_images/test"

classes = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
print("Classes:", classes)

# ------------------------------------------------------------
# AUGMENTATION (STRONG FOR 95%+)
# ------------------------------------------------------------
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(25),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.25, contrast=0.25),
    transforms.RandomAffine(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ------------------------------------------------------------
# DATASETS & LOADER
# ------------------------------------------------------------
train_ds = datasets.ImageFolder(train_dir, transform=train_transform)
val_ds = datasets.ImageFolder(val_dir, transform=val_transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

dataloaders = {'train': train_loader, 'val': val_loader}

# ------------------------------------------------------------
# LOAD DENSENET121 (STRONG ACCURACY)
# ------------------------------------------------------------
model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Freeze all layers first
for param in model.features.parameters():
    param.requires_grad = False

# Unfreeze the LAST DENSE BLOCK
for param in model.features.denseblock4.parameters():
    param.requires_grad = True

# Change classifier to 4 classes
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 4)

model = model.to(device)

# ------------------------------------------------------------
# LOSS & OPTIMIZER
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()

# Smaller LR → *very high accuracy*
optimizer = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)

# ------------------------------------------------------------
# TRAINING FUNCTION (95%+)
# ------------------------------------------------------------
def train_model(model, criterion, optimizer, dataloaders,
                num_epochs=30, patience=6):

    best_acc = 0.0
    best_wts = copy.deepcopy(model.state_dict())
    no_improve = 0

    for epoch in range(1, num_epochs + 1):
        start = time.time()
        print(f"\nEpoch {epoch}/{num_epochs}")
        print("-----------------------------------")

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0
            running_correct = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(preds == labels)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_correct.double() / len(dataloaders[phase].dataset)

            if phase == "train":
                print(f"Train Loss: {epoch_loss:.4f}")
            else:
                print(f"Val Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc*100:.2f}%")

                # Save best model
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_wts = copy.deepcopy(model.state_dict())
                    torch.save(model.state_dict(), "best_densenet_mri.pth")
                    print("Best model saved!")
                    no_improve = 0
                else:
                    no_improve += 1
                    print(f"⏳ No improvement. Early stopping in {patience - no_improve} epochs.")

        print(f"⏱ Time: {time.time() - start:.2f} sec")

        if no_improve >= patience:
            print("\nEarly Stopping Activated!")
            break

    print("\n Training Completed!")
    print("Best Model Path: best_densenet_mri.pth")

    model.load_state_dict(best_wts)
    return model


# ------------------------------------------------------------
# TRAIN MODEL
# ------------------------------------------------------------
model = train_model(model, criterion, optimizer, dataloaders)




 Device: cpu
Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\15016/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


100.0%



Epoch 1/30
-----------------------------------
Train Loss: 1.1883
Val Loss: 1.2936, Accuracy: 43.40%
Best model saved!
⏱ Time: 291.91 sec

Epoch 2/30
-----------------------------------
Train Loss: 0.8839
Val Loss: 1.2147, Accuracy: 47.97%
Best model saved!
⏱ Time: 287.84 sec

Epoch 3/30
-----------------------------------
Train Loss: 0.7222
Val Loss: 1.1705, Accuracy: 52.03%
Best model saved!
⏱ Time: 288.79 sec

Epoch 4/30
-----------------------------------
Train Loss: 0.6343
Val Loss: 1.1357, Accuracy: 54.57%
Best model saved!
⏱ Time: 277.59 sec

Epoch 5/30
-----------------------------------
Train Loss: 0.5553
Val Loss: 1.1366, Accuracy: 53.30%
⏳ No improvement. Early stopping in 5 epochs.
⏱ Time: 269.33 sec

Epoch 6/30
-----------------------------------
Train Loss: 0.5132
Val Loss: 1.1278, Accuracy: 56.35%
Best model saved!
⏱ Time: 382.65 sec

Epoch 7/30
-----------------------------------
Train Loss: 0.4911
Val Loss: 1.1553, Accuracy: 56.60%
Best model saved!
⏱ Time: 298.04 sec